# WASM demo - brainfuck
Brainfuck is an esoteric language that consists of only eight simple commands:

* `>` &nbsp;&nbsp; increment the data pointer (to point to the next cell to the right).
* `<` &nbsp;&nbsp; decrement the data pointer (to point to the next cell to the left).
* `+` &nbsp;&nbsp; increment (increase by one) the byte at the data pointer.
* `-` &nbsp;&nbsp; decrement (decrease by one) the byte at the data pointer.
* `.` &nbsp;&nbsp; output the byte at the data pointer.
* `,` &nbsp;&nbsp; accept one byte of input, storing its value in the byte at the data pointer.
* `[` &nbsp;&nbsp; if the byte at the data pointer is zero, then instead of moving the instruction pointer forward to the next command, jump it forward to the command after the matching ] command.
* `]` &nbsp;&nbsp; if the byte at the data pointer is nonzero, then instead of moving the instruction pointer forward to the next command, jump it back to the command after the matching [ command

Brainfuck is a simple language, but that does not mean that programming Brainfuck is easy!

In [ ]:
import wasmfun as wf

In [ ]:
def _commands2instructions(commands):
    """ Compile brainfuck commands to WASM instructions (as tuples).
    """    
    instructions = []
    while commands:
        c = commands.pop(0)
        if c == '>':
            instructions += [('get_local', 0), ('i32.const', 1), ('i32.add'), ('set_local', 0)]
        elif c == '<':
            instructions += [('get_local', 0), ('i32.const', 1), ('i32.sub'), ('set_local', 0)]
        elif c == '+':
            instructions += [('get_local', 0), ('get_local', 0),  # once for the read, once for the write
                             ('i32.load8_u', 0, 0),
                             ('i32.const', 1), ('i32.add'), ('i32.store8', 0, 0)]
        elif c == '-':
            instructions += [('get_local', 0), ('get_local', 0),  # once for the read, once for the write
                             ('i32.load8_u', 0, 0),
                             ('i32.const', 1), ('i32.sub'), ('i32.store8', 0, 0)]
        elif c == '.':
            instructions += [('get_local', 0), ('i32.load8_u', 0, 0), ('call', 0)]
        elif c == ',':
            # We don't support input, just set to zero
            instructions += [('get_local', 0), ('i32.const', 0), ('i32.store8', 0, 0)]
        elif c == '[':
            instructions += [('block', 'emptyblock'),
                                # if current data point == 0 goto end of block
                                ('get_local', 0), ('i32.load8_u', 0, 0), ('i32.const', 0), ('i32.eq'), ('br_if', 0),
                                ('loop', 'emptyblock'),
                                    ] + _commands2instructions(commands ) + [
                                    # if current data point > 0 goto start of block
                                    ('get_local', 0), ('i32.load8_u', 0, 0), ('i32.const', 0), ('i32.ne'), ('br_if', 0),
                                ('end'),
                             ('end')]
        elif c == ']':
            break
        else:
            raise ValueError('Unknown Brainfuck command: %r' % c)
    
    return instructions

## Hello world example

In [ ]:
EXAMPLE1 = """
[ This program prints "Hello World!" and a newline to the screen, its
  length is 106 active command characters. [It is not the shortest.]
  This loop is an "initial comment loop", a simple way of adding a comment
  to a BF program such that you don't have to worry about any command
  characters. Any ".", ",", "+", "-", "<" and ">" characters are simply
  ignored, the "[" and "]" characters just have to be balanced. This
  loop and the commands it contains are ignored because the current cell
  defaults to a value of 0; the 0 value causes this loop to be skipped.
]
++++++++               Set Cell #0 to 8
[
    >++++               Add 4 to Cell #1; this will always set Cell #1 to 4
    [                   as the cell will be cleared by the loop
        >++             Add 2 to Cell #2
        >+++            Add 3 to Cell #3
        >+++            Add 3 to Cell #4
        >+              Add 1 to Cell #5
        <<<<-           Decrement the loop counter in Cell #1
    ]                   Loop till Cell #1 is zero; number of iterations is 4
    >+                  Add 1 to Cell #2
    >+                  Add 1 to Cell #3
    >-                  Subtract 1 from Cell #4
    >>+                 Add 1 to Cell #6
    [<]                 Move back to the first zero cell you find; this will
                        be Cell #1 which was cleared by the previous loop
    <-                  Decrement the loop Counter in Cell #0
]                       Loop till Cell #0 is zero; number of iterations is 8
The result of this is:
Cell No :   0   1   2   3   4   5   6
Contents:   0   0  72 104  88  32   8
Pointer :   ^
>>.                     Cell #2 has value 72 which is 'H'
>---.                   Subtract 3 from Cell #3 to get 101 which is 'e'
+++++++..+++.           Likewise for 'llo' from Cell #3
>>.                     Cell #5 is 32 for the space
<-.                     Subtract 1 from Cell #4 for 87 to give a 'W'
<.                      Cell #3 was set to 'o' from the end of 'Hello'
+++.------.--------.    Cell #3 for 'rl' and 'd'
>>+.                    Add 1 to Cell #5 gives us an exclamation point
>++.                    And finally a newline from Cell #6
"""

In [ ]:
instructions = _commands2instructions([c for c in EXAMPLE1 if c in '><+-.,[]'])

In [ ]:
m = wf.Module(
    wf.ImportedFuncion('print_charcode', ['i32'], [], 'js', 'print_charcode'),
    wf.Function('$main', [], [], ['i32'], instructions),
    wf.MemorySection((1, 1)),
    wf.DataSection(),
    )
m.show()

In [ ]:
m.to_bytes()

In [ ]:
wf.run_wasm_in_notebook(m)

# Fibonacci example

In [ ]:
EXAMPLE2 = """
[Generate the fibonacci number sequence, (for numbers under 100). Taken from
http://esoteric.sange.fi/brainfuck/bf-source/prog/fibonacci.txt
]
+++++++++++ number of digits to output
> #1
+ initial number
>>>> #5
++++++++++++++++++++++++++++++++++++++++++++ (comma)
> #6
++++++++++++++++++++++++++++++++ (space)
<<<<<< #0
[
  > #1
  copy #1 to #7
  [>>>>>>+>+<<<<<<<-]>>>>>>>[<<<<<<<+>>>>>>>-]

  <
  divide #7 by 10 (begins in #7)
  [
    >
    ++++++++++  set the divisor #8
    [
      subtract from the dividend and divisor
      -<-
      if dividend reaches zero break out
        copy dividend to #9
        [>>+>+<<<-]>>>[<<<+>>>-]
        set #10
        +
        if #9 clear #10
        <[>[-]<[-]]
        if #10 move remaining divisor to #11
        >[<<[>>>+<<<-]>>[-]]
      jump back to #8 (divisor possition)
      <<
    ]
    if #11 is empty (no remainder) increment the quotient #12
    >>> #11
    copy to #13
    [>>+>+<<<-]>>>[<<<+>>>-]
    set #14
    +
    if #13 clear #14
    <[>[-]<[-]]
    if #14 increment quotient
    >[<<+>>[-]]
    <<<<<<< #7
  ]

  quotient is in #12 and remainder is in #11
  >>>>> #12
  if #12 output value plus offset to ascii 0
  [++++++++++++++++++++++++++++++++++++++++++++++++.[-]]
  subtract #11 from 10
  ++++++++++  #12 is now 10
  < #11
  [->-<]
  > #12
  output #12 even if it's zero
  ++++++++++++++++++++++++++++++++++++++++++++++++.[-]
  <<<<<<<<<<< #1

  check for final number
  copy #0 to #3
  <[>>>+>+<<<<-]>>>>[<<<<+>>>>-]
  <- #3
  if #3 output (comma) and (space)
  [>>.>.<<<[-]]
  << #1

  [>>+>+<<<-]>>>[<<<+>>>-]<<[<+>-]>[<+>-]<<<-
]
"""

In [ ]:
instructions = _commands2instructions([c for c in EXAMPLE2 if c in '><+-.,[]'])

In [ ]:
m = wf.Module(
    wf.ImportedFuncion('print_charcode', ['i32'], [], 'js', 'print_charcode'),
    wf.Function('$main', [], [], ['i32'], instructions),
    wf.MemorySection((1, 1)),
    wf.DataSection(),
    )
m.show()

In [ ]:
wf.run_wasm_in_notebook(m)

In [ ]:
wf.run_wasm_in_node(m)